# Properties of virtual vectors

Created 06/04/2024

Objectives:
* Explore and construct hypotheses around observations made in the transfer_matrix_class notebook.
* Test hypotheses across multiple wavefunctions and widths of the symmetry operation interval.
* What can be said about the left and right projected states of the bulk symmetry operation? Seems we can get extra information based by keeping the upper and lower legs separate.
    * How does the gauge impact this? Can include schmidt terms or not.
    * How does this relate to the principal eigenvector Pollman constructs to find a projective representation of the symmetry group?
    * How do these properties depend on the bond dimension?
* There are degeneracies in the schmidt terms. Is this just Pollman's entanglement entropy degeneracy? Does this impact the other observations?
* When acting with the transfer matrices derived from unitaries on these states, are there extra requirements?
    * Vector norm less than or equal to one?
    * Is there only a subspace of the entire space accessible?
    * Compare transfer matrices from unitaries with more general operators. What properties do transfer matrices from unitaries have?

# Package imports

In [1]:
from functools import reduce

import numpy as np
import pandas as pd

In [2]:
import h5py
from tenpy.tools import hdf5_io
import tenpy
import tenpy.linalg.np_conserved as npc

import os

In [12]:
from SPTOptimization.utils import (
    get_transfer_matrices_from_unitary_list,
    multiply_transfer_matrices
)

# Load data

In [3]:
DATA_DIR = r"data/transverse_cluster_200_site_dmrg"

In [4]:
loaded_data = list()

for local_file_name in os.listdir(DATA_DIR):
    f_name = r"{}/{}".format(DATA_DIR, local_file_name, ignore_unknown=False)
    with h5py.File(f_name, 'r') as f:
        data = hdf5_io.load_from_hdf5(f)
        loaded_data.append(data)

In [5]:
b_parameters = sorted(list(d['paramters']['B'] for d in loaded_data))

In [6]:
psi_dict = dict()

In [7]:
for b in b_parameters:
    psi = next(
        d['wavefunction']
        for d in loaded_data
        if d['paramters']['B'] == b
    )

    rounded_b = round(b, 1)
    psi_dict[rounded_b] = psi

In [8]:
list(psi_dict)

[0.0,
 0.1,
 0.2,
 0.3,
 0.4,
 0.5,
 0.6,
 0.7,
 0.8,
 0.9,
 1.0,
 1.1,
 1.2,
 1.3,
 1.4,
 1.5,
 1.6,
 1.7,
 1.8,
 1.9,
 2.0]

In [9]:
test_psi = psi_dict[0.5]

# Definitions

In [15]:
np_I = np.array([[1,0],[0,1]])
np_X = np.array([[0,1],[1,0]])
np_Y = np.array([[0,-1j],[1j,0]])
np_Z = np.array([[1,0],[0,-1]])

In [13]:
def get_symmetry_tm(psi, symmetry_operations):
    index = (psi.L - len(symmetry_operations))//2

    tms = get_transfer_matrices_from_unitary_list(
        psi,
        symmetry_operations,
        index
    )

    transfer_matrix = reduce(
        multiply_transfer_matrices,
        tms
    )

    np_transfer_matrix = (
        transfer_matrix
        .combine_legs([['vL', 'vL*'], ['vR', 'vR*']])
        .to_ndarray()
    )

    return np_transfer_matrix

# Code

## Schmidt values

In [22]:
SR_values = [psi.get_SR(100) for psi in psi_dict.values()]

Increase B until critical point (SPT phase)

In [23]:
SR_values[0]

array([0.70710678, 0.70710678])

In [30]:
SR_values[2]

array([7.07102179e-01, 7.07102179e-01, 1.80398424e-03, 1.80398424e-03,
       1.80398424e-03, 1.80398424e-03, 4.60238879e-06, 4.60238879e-06])

In [31]:
SR_values[4]

array([7.07022994e-01, 7.07022994e-01, 7.69671387e-03, 7.69671387e-03,
       7.69671387e-03, 7.69671387e-03, 8.37870972e-05, 8.37870972e-05])

In [32]:
SR_values[6]

array([7.06564503e-01, 7.06564503e-01, 1.95743348e-02, 1.95743348e-02,
       1.95743348e-02, 1.95743348e-02, 5.42278277e-04, 5.42278277e-04])

In [33]:
SR_values[8]

array([0.7044495 , 0.70443499, 0.04332492, 0.04332492, 0.04332402,
       0.04332402, 0.00266456, 0.00266451])

In [34]:
SR_values[9]

array([0.70341105, 0.69848145, 0.06592236, 0.06591902, 0.06546036,
       0.06545705, 0.00617781, 0.00613451])

Critical point

In [26]:
SR_values[10]

array([0.89538113, 0.40599408, 0.15866992, 0.06737791, 0.05457546,
       0.02441625, 0.01049789, 0.00778904])

Continue increasing B (trivial phase)

In [29]:
SR_values[11]

array([0.90859608, 0.28759973, 0.28757468, 0.09101176, 0.01856608,
       0.01855351, 0.00587403, 0.00581269])

In [36]:
SR_values[13]

array([0.94755227, 0.22269151, 0.22269149, 0.05233448, 0.01027601,
       0.01027562, 0.00241477, 0.00240155])

In [37]:
SR_values[15]

array([0.9646692 , 0.18451223, 0.18451223, 0.03529132, 0.00614738,
       0.00614728, 0.00117573, 0.00117247])

In [38]:
SR_values[17]

array([9.74172204e-01, 1.58571627e-01, 1.58571627e-01, 2.58115448e-02,
       3.97020923e-03, 3.97017911e-03, 6.46226331e-04, 6.45246442e-04])

In [39]:
SR_values[20]

array([9.82343696e-01, 1.31678800e-01, 1.31678800e-01, 1.76509457e-02,
       2.29198365e-03, 2.29197709e-03, 3.07223721e-04, 3.07005852e-04])

### Conclusions
* There is a natrual (2,4,2) grouping of the schmidt values in the non-trivial phase. Not sure about the 4 group but regardless of bond dimension (provided it's even...) can we conclude that there is a general two fold degeneraxy?
    * This degeneracy breaks down as we approach the critical point. This could be due to numerical errors due to the long correlation length.
* In the non-trivial phase, the largest two schmidt values dominate for low B, resulting in a value of $2^{-1/2}$. The lower values increase with incresing B. Makes sense, a more complicated state needs a higher effective bond dimension.
* In the "trivial" phase:
    * The largest schmidt value is unique and tends to 1 with increasing B.
    * There is a natural (1, 2, 1, 2, 2) grouping. Weird!

## Transfer matrix singular values and eigenvalues

In [19]:
symmetry_ops = [
    [np_I, np_I]*40,
    [np_X, np_I]*40,
    [np_I, np_X]*40,
    [np_X, np_X]*40
]

### B normalisation
Calcualte the transfer matrices (in the "B" normalisation).

In [20]:
symmetry_tms = [
    [get_symmetry_tm(psi, ops) for ops in symmetry_ops]
    for psi in psi_dict.values()
]   

In [79]:
def singular_and_eigen_values(transfer_matrix, smallest=1e-4, num_digits=6):
    singular_values = np.linalg.svd(transfer_matrix).S
    filtered_singular_values = np.round(
        singular_values[singular_values >= smallest],
        num_digits
    )

    eigen_values = np.linalg.eig(transfer_matrix).eigenvalues
    filtered_eigen_values = np.round(
        eigen_values[np.abs(eigen_values) >= smallest],
        num_digits
    )

    return (filtered_singular_values, filtered_eigen_values)

#### B=0

In [97]:
index=0

In [98]:
(
    singular_and_eigen_values(symmetry_tms[index][0]),
    singular_and_eigen_values(symmetry_tms[index][1]),
    singular_and_eigen_values(symmetry_tms[index][2]),
    singular_and_eigen_values(symmetry_tms[index][3])
)

((array([1.]), array([1.+0.j])),
 (array([1.]), array([0.990918+0.j])),
 (array([1.]), array([-0.990918+0.j])),
 (array([1.]), array([-1.+0.j])))

#### B=0.2

In [54]:
index=2

In [72]:
(
    singular_and_eigen_values(symmetry_tms[index][0]),
    singular_and_eigen_values(symmetry_tms[index][1]),
    singular_and_eigen_values(symmetry_tms[index][2]),
    singular_and_eigen_values(symmetry_tms[index][3])
)

((array([1.999974]), array([1.+0.j])),
 (array([1.999974]), array([-0.626357+0.j])),
 (array([1.999974]), array([-0.626365+0.j])),
 (array([1.999974]), array([0.999984+0.j])))

#### B=0.4

In [75]:
index=4

In [76]:
(
    singular_and_eigen_values(symmetry_tms[index][0]),
    singular_and_eigen_values(symmetry_tms[index][1]),
    singular_and_eigen_values(symmetry_tms[index][2]),
    singular_and_eigen_values(symmetry_tms[index][3])
)

((array([1.999526]), array([1.+0.j])),
 (array([1.999526]), array([0.122726+0.j])),
 (array([1.999526]), array([-0.12273+0.j])),
 (array([1.999526]), array([-0.999989+0.j])))

#### B=0.5

In [73]:
index=5

In [74]:
(
    singular_and_eigen_values(symmetry_tms[index][0]),
    singular_and_eigen_values(symmetry_tms[index][1]),
    singular_and_eigen_values(symmetry_tms[index][2]),
    singular_and_eigen_values(symmetry_tms[index][3])
)

((array([1.998715]), array([1.+0.j])),
 (array([1.998715]), array([-0.000366+0.j])),
 (array([1.998715]), array([-0.000366+0.j])),
 (array([1.998715]), array([0.999357+0.j])))

#### B=0.7

In [80]:
index=7
smallest = 1e-6

In [81]:
(
    singular_and_eigen_values(symmetry_tms[index][0], smallest=smallest),
    singular_and_eigen_values(symmetry_tms[index][1], smallest=smallest),
    singular_and_eigen_values(symmetry_tms[index][2], smallest=smallest),
    singular_and_eigen_values(symmetry_tms[index][3], smallest=smallest)
)

((array([1.99316]), array([1.+0.j])),
 (array([1.99316]), array([0.000252+0.j])),
 (array([1.99316]), array([-0.000251+0.j])),
 (array([1.99316]), array([-0.99316+0.j])))

#### B=0.9

In [86]:
index=9
smallest = 1e-4

In [87]:
(
    singular_and_eigen_values(symmetry_tms[index][0], smallest=smallest),
    singular_and_eigen_values(symmetry_tms[index][1], smallest=smallest),
    singular_and_eigen_values(symmetry_tms[index][2], smallest=smallest),
    singular_and_eigen_values(symmetry_tms[index][3], smallest=smallest)
)

((array([1.966918e+00, 3.730000e-04, 3.720000e-04]), array([1.+0.j])),
 (array([1.966716e+00, 3.730000e-04, 3.720000e-04]), array([0.040828-0.j])),
 (array([1.963493e+00, 3.730000e-04, 3.720000e-04]), array([0.040825-0.j])),
 (array([1.963291e+00, 3.730000e-04, 3.720000e-04]), array([0.981271-0.j])))

#### B=1.0

In [88]:
index=10
smallest = 1e-4

In [89]:
(
    singular_and_eigen_values(symmetry_tms[index][0], smallest=smallest),
    singular_and_eigen_values(symmetry_tms[index][1], smallest=smallest),
    singular_and_eigen_values(symmetry_tms[index][2], smallest=smallest),
    singular_and_eigen_values(symmetry_tms[index][3], smallest=smallest)
)

((array([2.315801, 0.004263]), array([1.      +0.j, 0.001672+0.j])),
 (array([2.315799, 0.004263]), array([0.999999-0.j, 0.001672+0.j])),
 (array([0.160169, 0.135437]), array([0.058347+0.j, 0.081438+0.j])),
 (array([0.160161, 0.135443]), array([0.058349+0.j, 0.081436+0.j])))

#### B=1.1

In [90]:
index=11
smallest = 1e-4

In [91]:
(
    singular_and_eigen_values(symmetry_tms[index][0], smallest=smallest),
    singular_and_eigen_values(symmetry_tms[index][1], smallest=smallest),
    singular_and_eigen_values(symmetry_tms[index][2], smallest=smallest),
    singular_and_eigen_values(symmetry_tms[index][3], smallest=smallest)
)

((array([2.35587e+00, 3.78000e-04, 1.55000e-04]),
  array([ 1.00e+00+0.j, -1.67e-04+0.j])),
 (array([2.35587e+00, 3.78000e-04, 1.55000e-04]),
  array([ 9.99909e-01+0.j, -1.67000e-04+0.j])),
 (array([2.35587e+00, 3.78000e-04, 1.55000e-04]),
  array([ 9.99909e-01+0.j, -1.67000e-04+0.j])),
 (array([2.35587e+00, 3.78000e-04, 1.55000e-04]),
  array([ 1.00e+00+0.j, -1.67e-04+0.j])))

#### B=1.5

In [93]:
index=15
smallest = 1e-4

In [94]:
(
    singular_and_eigen_values(symmetry_tms[index][0], smallest=smallest),
    singular_and_eigen_values(symmetry_tms[index][1], smallest=smallest),
    singular_and_eigen_values(symmetry_tms[index][2], smallest=smallest),
    singular_and_eigen_values(symmetry_tms[index][3], smallest=smallest)
)

((array([2.635619]), array([1.+0.j])),
 (array([2.635619]), array([1.+0.j])),
 (array([2.635619]), array([1.+0.j])),
 (array([2.635619]), array([1.+0.j])))

#### B=2.0

In [95]:
index=20
smallest = 1e-4

In [96]:
(
    singular_and_eigen_values(symmetry_tms[index][0], smallest=smallest),
    singular_and_eigen_values(symmetry_tms[index][1], smallest=smallest),
    singular_and_eigen_values(symmetry_tms[index][2], smallest=smallest),
    singular_and_eigen_values(symmetry_tms[index][3], smallest=smallest)
)

((array([2.730311]), array([1.+0.j])),
 (array([2.730311]), array([1.+0.j])),
 (array([2.730311]), array([1.+0.j])),
 (array([2.730311]), array([1.+0.j])))

### Symmetric normalisation
Incorporate the schmidt values on the left hand side. Should make the left and right regions symmetric.

In [103]:
def get_symmetric_symmetry_tm(psi, symmetry_operations):
    index = (psi.L - len(symmetry_operations))//2

    tms = get_transfer_matrices_from_unitary_list(
        psi,
        symmetry_operations,
        index
    )

    transfer_matrix = reduce(
        multiply_transfer_matrices,
        tms
    )

    left_leg = psi.get_B(index).legs[0]
    SL = npc.diag(psi.get_SL(index), left_leg, labels = ['vL', 'vR'])
    
    transfer_matrix = npc.tensordot(
        SL,
        transfer_matrix,
        [['vR',],  ['vL',]]
    )
    transfer_matrix = npc.tensordot(
        SL.conj(),
        transfer_matrix,
        [['vR*',],  ['vL*',]]
    )
    
    np_transfer_matrix = (
        transfer_matrix
        .combine_legs([['vL', 'vL*'], ['vR', 'vR*']])
        .to_ndarray()
    )

    return np_transfer_matrix

In [104]:
symmetric_symmetry_tms = [
    [get_symmetric_symmetry_tm(psi, ops) for ops in symmetry_ops]
    for psi in psi_dict.values()
]   

#### B=0

In [113]:
index=0

In [114]:
(
    singular_and_eigen_values(symmetric_symmetry_tms[index][0]),
    singular_and_eigen_values(symmetric_symmetry_tms[index][1]),
    singular_and_eigen_values(symmetric_symmetry_tms[index][2]),
    singular_and_eigen_values(symmetric_symmetry_tms[index][3])
)

((array([0.5]), array([0.5+0.j])),
 (array([0.5]), array([0.495459+0.j])),
 (array([0.5]), array([-0.495459+0.j])),
 (array([0.5]), array([-0.5+0.j])))

#### B=0.2

In [115]:
index=2

In [116]:
(
    singular_and_eigen_values(symmetric_symmetry_tms[index][0]),
    singular_and_eigen_values(symmetric_symmetry_tms[index][1]),
    singular_and_eigen_values(symmetric_symmetry_tms[index][2]),
    singular_and_eigen_values(symmetric_symmetry_tms[index][3])
)

((array([0.499987]), array([0.499987+0.j])),
 (array([0.499987]), array([-0.313174+0.j])),
 (array([0.499987]), array([-0.313174+0.j])),
 (array([0.499987]), array([0.499987+0.j])))

#### B=0.4

In [117]:
index=4

In [118]:
(
    singular_and_eigen_values(symmetric_symmetry_tms[index][0]),
    singular_and_eigen_values(symmetric_symmetry_tms[index][1]),
    singular_and_eigen_values(symmetric_symmetry_tms[index][2]),
    singular_and_eigen_values(symmetric_symmetry_tms[index][3])
)

((array([0.499763]), array([0.499763+0.j])),
 (array([0.499763]), array([0.061354+0.j])),
 (array([0.499763]), array([-0.061354+0.j])),
 (array([0.499763]), array([-0.499763+0.j])))

#### B=0.5

In [119]:
index=5

In [120]:
(
    singular_and_eigen_values(symmetric_symmetry_tms[index][0]),
    singular_and_eigen_values(symmetric_symmetry_tms[index][1]),
    singular_and_eigen_values(symmetric_symmetry_tms[index][2]),
    singular_and_eigen_values(symmetric_symmetry_tms[index][3])
)

((array([0.499358]), array([0.499358+0.j])),
 (array([0.499358]), array([-0.000183+0.j])),
 (array([0.499358]), array([-0.000183+0.j])),
 (array([0.499358]), array([0.499358+0.j])))

#### B=0.7

In [121]:
index=7
smallest = 1e-6

In [122]:
(
    singular_and_eigen_values(symmetric_symmetry_tms[index][0], smallest=smallest),
    singular_and_eigen_values(symmetric_symmetry_tms[index][1], smallest=smallest),
    singular_and_eigen_values(symmetric_symmetry_tms[index][2], smallest=smallest),
    singular_and_eigen_values(symmetric_symmetry_tms[index][3], smallest=smallest)
)

((array([0.496586]), array([0.496586+0.j])),
 (array([0.496586]), array([0.000125+0.j])),
 (array([0.496586]), array([-0.000125-0.j])),
 (array([0.496586]), array([-0.49658+0.j])))

#### B=0.9

In [123]:
index=9
smallest = 1e-4

In [124]:
(
    singular_and_eigen_values(symmetric_symmetry_tms[index][0], smallest=smallest),
    singular_and_eigen_values(symmetric_symmetry_tms[index][1], smallest=smallest),
    singular_and_eigen_values(symmetric_symmetry_tms[index][2], smallest=smallest),
    singular_and_eigen_values(symmetric_symmetry_tms[index][3], smallest=smallest)
)

((array([0.483596]), array([0.483596+0.j])),
 (array([0.482753]), array([0.020243+0.j])),
 (array([0.482753]), array([0.020243+0.j])),
 (array([0.481912]), array([0.481834+0.j])))

#### B=1.0

In [125]:
index=10
smallest = 1e-4

In [126]:
(
    singular_and_eigen_values(symmetric_symmetry_tms[index][0], smallest=smallest),
    singular_and_eigen_values(symmetric_symmetry_tms[index][1], smallest=smallest),
    singular_and_eigen_values(symmetric_symmetry_tms[index][2], smallest=smallest),
    singular_and_eigen_values(symmetric_symmetry_tms[index][3], smallest=smallest)
)

((array([6.70405e-01, 3.80000e-04]),
  array([6.70405e-01+0.j, 3.80000e-04+0.j])),
 (array([6.70404e-01, 3.80000e-04]),
  array([6.70404e-01+0.j, 3.80000e-04+0.j])),
 (array([0.026992, 0.026135]), array([0.026992+0.j, 0.026135+0.j])),
 (array([0.026982, 0.026145]), array([0.026982+0.j, 0.026144-0.j])))

#### B=1.1

In [127]:
index=11
smallest = 1e-4

In [128]:
(
    singular_and_eigen_values(symmetric_symmetry_tms[index][0], smallest=smallest),
    singular_and_eigen_values(symmetric_symmetry_tms[index][1], smallest=smallest),
    singular_and_eigen_values(symmetric_symmetry_tms[index][2], smallest=smallest),
    singular_and_eigen_values(symmetric_symmetry_tms[index][3], smallest=smallest)
)

((array([0.694509]), array([0.694508+0.j])),
 (array([0.694509]), array([0.694508+0.j])),
 (array([0.694509]), array([0.694508+0.j])),
 (array([0.694509]), array([0.694508+0.j])))

#### B=1.5

In [129]:
index=15
smallest = 1e-4

In [130]:
(
    singular_and_eigen_values(symmetric_symmetry_tms[index][0], smallest=smallest),
    singular_and_eigen_values(symmetric_symmetry_tms[index][1], smallest=smallest),
    singular_and_eigen_values(symmetric_symmetry_tms[index][2], smallest=smallest),
    singular_and_eigen_values(symmetric_symmetry_tms[index][3], smallest=smallest)
)

((array([0.868311]), array([0.868311+0.j])),
 (array([0.868311]), array([0.868311+0.j])),
 (array([0.868311]), array([0.868311+0.j])),
 (array([0.868311]), array([0.868311+0.j])))

#### B=2.0

In [131]:
index=20
smallest = 1e-4

In [132]:
(
    singular_and_eigen_values(symmetric_symmetry_tms[index][0], smallest=smallest),
    singular_and_eigen_values(symmetric_symmetry_tms[index][1], smallest=smallest),
    singular_and_eigen_values(symmetric_symmetry_tms[index][2], smallest=smallest),
    singular_and_eigen_values(symmetric_symmetry_tms[index][3], smallest=smallest)
)

((array([0.931825]), array([0.931825+0.j])),
 (array([0.931825]), array([0.931825+0.j])),
 (array([0.931825]), array([0.931825+0.j])),
 (array([0.931825]), array([0.931825+0.j])))

### Conclusions
* B gauge
    * Nontrivial phase
        * Dominant singular value is 2 and decreases with B, constant across symmetry operations. Is only 1 for the B=0 phase, implying bond dimension dependence.
        * The eigenvalues of the $II$ and $XX$ transfer matrices are 1. The eigenvalues for $IX$ and $XI$ are typically much smaller, and can change sign. These smaller eigenvalues seem to have an upward parabolic shape wrt B. 
    * Trivial phase
        * Dominant singular values increase with B and are fixed across symmetry operations.
        * The dominant eigenvalue is 1 consistently.
* Symmetric gauge
    * Nontrivial phase
        * Dominant singular value is typically 0.5, slightly decreasing with B (although this could be an artifact of increasing correlation length).
        * The eigenvalues of the $II$ and $XX$ transfer matrices agree with the dominant singular value. The eigenvalues for $IX$ and $XI$ are typically much smaller, and can change sign. These smaller eigenvalues seem to have an upward parabolic shape wrt B.    
    * Trivial phase
        * There is a single dominant singular value and eigenvalue, and they agree across all symmetry operations.
        * This value increases with B. Tends towards 1 for perfect product state?

## Symmetry projected states
The dominant singular value and eigenvalue are non-degenerate. So can find eigenvectors and SVD decomposition, and investigate the structure of these vectors wrt the upper and lower legs.

Check how well this approximates the original transfer matrix.

In [20]:
symmetry_tms = [
    [get_symmetry_tm(psi, ops) for ops in symmetry_ops]
    for psi in psi_dict.values()
]   

In [140]:
def reshape_1D_array_to_square(a):
    assert len(a.shape) == 1
    N = len(a)

    sqrt_N = np.sqrt(N)
    assert int(sqrt_N)**2 == N
    sqrt_N = int(sqrt_N)

    return a.reshape((sqrt_N, sqrt_N))

In [141]:
def singular_and_eigen_vectors(transfer_matrix):
    U, S, Vh = np.linalg.svd(transfer_matrix)
    left_projected_state = U[:, 0]
    right_projected_state = Vh[0]

    svd_approx_tm = S[0]*np.outer(left_projected_state, right_projected_state)
    svd_approx_score = np.linalg.norm(svd_approx_tm - transfer_matrix)

    eig_vals, eig_vecs = np.linalg.eig(transfer_matrix)
    dom_eig_index = np.argmax(np.abs(eig_vals))
    
    dom_eig_vec = eig_vecs[dom_eig_index]
    dom_eig_val = eig_vals[dom_eig_index]
    eig_approx_tm = dom_eig_val*np.outer(dom_eig_vec, dom_eig_vec.conj())
    eig_approx_score = np.linalg.norm(eig_approx_tm - transfer_matrix)

    reshape_and_svd_eig_vals = (
        lambda x: singular_and_eigen_values(reshape_1D_array_to_square(x))
    )

    out = (
        svd_approx_score,
        reshape_and_svd_eig_vals(right_projected_state),
        reshape_and_svd_eig_vals(left_projected_state),
        eig_approx_score,
        reshape_and_svd_eig_vals(dom_eig_vec),
    )

    return out

In [145]:
singular_and_eigen_vectors_data = [
    [singular_and_eigen_vectors(tm) for tm in l]
    for l in symmetry_tms
]

#### SVD approximations

In [146]:
svd_approx_scores = np.zeros((len(singular_and_eigen_vectors_data), 4))

In [156]:
for i, t in enumerate(singular_and_eigen_vectors_data):
    for j, l in enumerate(t):
        svd_approx_scores[i, j] = l[0]

In [157]:
svd_approx_scores

array([[5.36346694e-16, 5.59702084e-16, 4.67068457e-17, 1.11537779e-16],
       [4.85164237e-16, 5.74989589e-16, 6.87166274e-16, 4.53175722e-16],
       [5.24641080e-16, 6.41149517e-16, 7.53849329e-16, 9.79082987e-16],
       [3.73013560e-16, 1.19932635e-15, 5.53236376e-16, 8.76038272e-16],
       [7.17313986e-16, 4.17451905e-16, 5.24113563e-16, 1.24494527e-15],
       [1.84419317e-14, 1.84443693e-14, 1.84457972e-14, 1.84423143e-14],
       [2.69964150e-11, 2.69964152e-11, 2.69964152e-11, 2.69964153e-11],
       [1.39288730e-08, 1.39288730e-08, 1.39288730e-08, 1.39288730e-08],
       [3.47279683e-06, 3.47279690e-06, 3.47280170e-06, 3.47280178e-06],
       [5.26242950e-04, 5.26297073e-04, 5.27161092e-04, 5.27215339e-04],
       [4.26297063e-03, 4.26296582e-03, 1.35436740e-01, 1.35443008e-01],
       [4.08047998e-04, 4.08047998e-04, 4.08047998e-04, 4.08047998e-04],
       [1.68318028e-05, 1.68318028e-05, 1.68318028e-05, 1.68318028e-05],
       [9.47095255e-07, 9.47095255e-07, 9.47095255e

In [158]:
np.max(svd_approx_scores)

0.13544300761679562

In [159]:
svd_approx_scores[9]

array([0.00052624, 0.0005263 , 0.00052716, 0.00052722])

In [160]:
svd_approx_scores[11]

array([0.00040805, 0.00040805, 0.00040805, 0.00040805])

So taking the largest singualar value for the symmetry transfer matrix is a good approximation apart from the critical point. It's possible it's also poor close to the critical point.

#### Eigenvalue approximations

In [161]:
eig_approx_scores = np.zeros((len(singular_and_eigen_vectors_data), 4))

In [165]:
for i, t in enumerate(singular_and_eigen_vectors_data):
    for j, l in enumerate(t):
        eig_approx_scores[i, j] = l[3]

In [166]:
eig_approx_scores

array([[1.22474487, 1.08852046, 1.24237517, 0.00783531],
       [1.97694077, 1.99204196, 1.99649203, 1.97246519],
       [1.97443489, 1.99245448, 2.01061473, 1.97342094],
       [1.99986111, 1.99750365, 1.98514497, 1.98332061],
       [1.9995595 , 2.00341564, 1.99894589, 1.99909804],
       [1.97156109, 1.99871487, 1.9987127 , 1.99809767],
       [1.96991453, 1.99693476, 1.99693091, 1.99612806],
       [1.96644909, 1.99316031, 1.99315802, 1.99367301],
       [1.95858028, 1.98498516, 1.98499184, 1.98371456],
       [1.93908453, 1.96670113, 1.96292795, 1.96150014],
       [2.26718141, 2.28272216, 0.20845138, 0.20853696],
       [2.31542812, 2.31540076, 2.31491434, 2.31486262],
       [2.4323773 , 2.43214797, 2.43248585, 2.43239125],
       [2.50600126, 2.50602338, 2.50603559, 2.50596088],
       [2.55693476, 2.5569266 , 2.55691837, 2.55694076],
       [2.59415558, 2.5941577 , 2.5941494 , 2.59414152],
       [2.62241432, 2.62243366, 2.62241249, 2.62239503],
       [2.64452331, 2.64452457,

The eigenvalue decomposition is universally bad.

#### Eigenvalues and eigenvectors of right projected states

In [170]:
1/np.sqrt(2)

0.7071067811865475

In [173]:
for i, t in enumerate(singular_and_eigen_vectors_data):
    print("\ni={}".format(i))
    
    for j, l in enumerate(t):
        print(l[1])


i=0
(array([0.707107, 0.707107]), array([-0.707107+0.j, -0.707107+0.j]))
(array([0.707107, 0.707107]), array([-0.707107+0.j,  0.707107+0.j]))
(array([0.707107, 0.707107]), array([-0.707107+0.j,  0.707107+0.j]))
(array([0.707107, 0.707107]), array([0.+0.707107j, 0.-0.707107j]))

i=1
(array([0.707107, 0.707107]), array([-0.707107+0.j, -0.707107+0.j]))
(array([0.707107, 0.707107]), array([-0.707107+0.j,  0.707107+0.j]))
(array([0.707107, 0.707107]), array([ 0.707107+0.j, -0.707107+0.j]))
(array([0.707107, 0.707107]), array([ 0.+0.707107j, -0.-0.707107j]))

i=2
(array([0.707107, 0.707107]), array([-0.707107+0.j, -0.707107+0.j]))
(array([0.707107, 0.707107]), array([ 0.707107+0.j, -0.707107+0.j]))
(array([0.707107, 0.707107]), array([-0.707107+0.j,  0.707107+0.j]))
(array([0.707107, 0.707107]), array([0.+0.707107j, 0.-0.707107j]))

i=3
(array([0.707107, 0.707107]), array([-0.707107+0.j, -0.707107+0.j]))
(array([0.707107, 0.707107]), array([-0.707107+0.j,  0.707107+0.j]))
(array([0.707107, 

##### Conclusions

* The dominant (all?) singular and eigenvalues agree, up to a possible sign flip of the eigenvalues.
* The singular values seem to be doubly degenerate at B=0, but this degeneracy splits as B goes to 1.
* Almost all virtual vectors seem to be well approximated by keeping the top two singular values. For values of B close to the critical point, this breaks down somewhat. In the trivial phase, only one singular value seems needed.
* The singular values sum to 1, so wether the number of dominant terms determines their value, provided the values are degenerate.
* The dominant singular value is going to 1 (slowly?) as B goes to infinity.
* Different symmetry actions can give different signs for the eigenvalues. Seems a bit arbitrary though....

#### Eigenvalues and eigenvectors of left projected states

In [174]:
for i, t in enumerate(singular_and_eigen_vectors_data):
    print("\ni={}".format(i))
    
    for j, l in enumerate(t):
        print(l[2])


i=0
(array([0.707107, 0.707107]), array([-0.707107+0.j, -0.707107+0.j]))
(array([0.707107, 0.707107]), array([-0.707107+0.j,  0.707107+0.j]))
(array([0.707107, 0.707107]), array([-0.707107+0.j,  0.707107+0.j]))
(array([0.707107, 0.707107]), array([ 0.+0.707107j, -0.-0.707107j]))

i=1
(array([0.353553, 0.353553, 0.353553, 0.353553, 0.353553, 0.353553,
       0.353553, 0.353553]), array([-0.353553+0.j, -0.353553+0.j, -0.353553+0.j, -0.353553+0.j,
       -0.353553+0.j, -0.353553+0.j, -0.353553+0.j, -0.353553+0.j]))
(array([0.353553, 0.353553, 0.353553, 0.353553, 0.353553, 0.353553,
       0.353553, 0.353553]), array([ 0.353553+0.j,  0.353553+0.j, -0.353553+0.j, -0.353553+0.j,
       -0.353553+0.j,  0.353553+0.j, -0.353553+0.j,  0.353553+0.j]))
(array([0.353553, 0.353553, 0.353553, 0.353553, 0.353553, 0.353553,
       0.353553, 0.353553]), array([-0.353553+0.j,  0.353553+0.j,  0.353553+0.j,  0.353553+0.j,
       -0.353553+0.j, -0.353553+0.j, -0.353553+0.j,  0.353553+0.j]))
(array([0.35355

##### Conclusions
* Getting the same dominant singular value and eigenvalue $2^{-3/2}$ with the same degeneracy of 8 across all values! Seems totally different to the right side.

#### Eigenvalues and eigenvectors of eigenvector

In [175]:
for i, t in enumerate(singular_and_eigen_vectors_data):
    print("\ni={}".format(i))
    
    for j, l in enumerate(t):
        print(l[4])


i=0
(array([1.]), array([0.707107+0.j]))
(array([1.010974, 0.166342]), array([-0.705847-0.j,  0.238249+0.j]))
(array([0.846805, 0.301102]), array([0.179242+0.472067j, 0.179242-0.472067j]))
(array([0.707109, 0.702574]), array([-0.000106+0.704838j, -0.000106-0.704838j]))

i=1
(array([0.427183]), array([0.353553-0.j]))
(array([0.496479]), array([0.346569-0.j]))
(array([0.380336]), array([-0.353134+0.j]))
(array([0.353554]), array([ 0.000157+0.j, -0.000157+0.j]))

i=2
(array([0.391514]), array([0.353553-0.j]))
(array([0.373913]), array([-0.352233-0.j]))
(array([5.96298e-01, 2.82000e-04]), array([-2.9623e-01-9.8e-05j, -1.1200e-04+1.8e-05j]))
(array([0.37364]), array([ 0.00082 +3.2e-05j, -0.000819-3.2e-05j]))

i=3
(array([0.003238]), array([], dtype=complex128))
(array([0.232303]), array([0.07672-4.5e-05j]))
(array([4.92254e-01, 1.01000e-04]), array([0.271141-2.8e-05j]))
(array([0.493094]), array([ 0.000798+0.002462j, -0.000799-0.002398j]))

i=4
(array([0.107498, 0.002111]), array([-0.00011

##### Conclusions
* Not much going on here...

#### Symmetric normalisation/gauge

In [176]:
singular_and_eigen_vectors_symmetric_data = [
    [singular_and_eigen_vectors(tm) for tm in l]
    for l in symmetric_symmetry_tms
]

#### SVD approximations

In [177]:
svd_approx_scores = np.zeros((len(singular_and_eigen_vectors_symmetric_data), 4))

In [178]:
for i, t in enumerate(singular_and_eigen_vectors_symmetric_data):
    for j, l in enumerate(t):
        svd_approx_scores[i, j] = l[0]

In [179]:
svd_approx_scores

array([[1.01425765e-16, 1.22977779e-16, 8.82076930e-17, 3.10254709e-16],
       [1.75896562e-16, 1.20630343e-16, 1.74526784e-16, 1.80435790e-16],
       [2.13526353e-16, 1.04386551e-16, 4.51904623e-16, 3.93211296e-16],
       [2.88419283e-16, 1.59256092e-16, 1.29583550e-16, 3.47378023e-16],
       [9.89916515e-17, 1.48611054e-16, 3.53562633e-16, 1.40677803e-16],
       [8.90445676e-16, 8.91243608e-16, 9.19543886e-16, 8.92484791e-16],
       [1.62556317e-12, 1.62556318e-12, 1.62556312e-12, 1.62556318e-12],
       [1.03555891e-09, 1.03555891e-09, 1.03555891e-09, 1.03555891e-09],
       [3.18500371e-07, 3.18500818e-07, 3.18500818e-07, 3.18501265e-07],
       [6.03738889e-05, 6.04792643e-05, 6.04792833e-05, 6.05848483e-05],
       [3.80206235e-04, 3.80205842e-04, 2.61348989e-02, 2.61445714e-02],
       [4.51681425e-05, 4.51681425e-05, 4.51681425e-05, 4.51681425e-05],
       [1.33359552e-06, 1.33359552e-06, 1.33359552e-06, 1.33359552e-06],
       [5.92182523e-08, 5.92182523e-08, 5.92182523e

In [180]:
np.max(svd_approx_scores)

0.026144571373785726

Better approximation than the B gauge case

#### Eigenvalue approximations

In [181]:
eig_approx_scores = np.zeros((len(singular_and_eigen_vectors_symmetric_data), 4))

In [182]:
for i, t in enumerate(singular_and_eigen_vectors_symmetric_data):
    for j, l in enumerate(t):
        eig_approx_scores[i, j] = l[3]

In [183]:
eig_approx_scores

array([[1.11803398, 0.54853895, 0.45332733, 0.70677572],
       [0.49999921, 0.51825955, 0.48977673, 0.54231527],
       [0.49998698, 0.80860483, 0.57935654, 0.61235659],
       [0.49993056, 0.47743812, 0.50535118, 0.47720153],
       [0.49976307, 0.53465117, 0.49799317, 0.50766764],
       [0.6158232 , 0.49935764, 0.49940312, 1.01410954],
       [0.60561947, 0.49846854, 0.49853272, 0.93265643],
       [0.60106037, 0.496586  , 0.49655527, 0.51923876],
       [0.59311984, 0.49252008, 0.49262221, 0.6722498 ],
       [0.55866312, 0.4826757 , 0.47816266, 0.47911258],
       [0.21209517, 0.21005576, 0.03269231, 0.03221352],
       [0.19435711, 0.20425163, 0.19673405, 0.20276928],
       [0.16015548, 0.10167415, 0.16114517, 0.18104475],
       [0.12447046, 0.12654298, 0.12686527, 0.09328179],
       [0.09547002, 0.10140607, 0.0781148 , 0.1237035 ],
       [0.08954414, 0.08992178, 0.08982612, 0.09625147],
       [0.07778092, 0.07739974, 0.0749607 , 0.05937821],
       [0.07017208, 0.064161  ,

Better, but still bad.

#### Eigenvalues and eigenvectors of right projected states

In [184]:
for i, t in enumerate(singular_and_eigen_vectors_symmetric_data):
    print("\ni={}".format(i))
    
    for j, l in enumerate(t):
        print(l[1])


i=0
(array([0.707107, 0.707107]), array([-0.707107+0.j, -0.707107+0.j]))
(array([0.707107, 0.707107]), array([-0.707107+0.j,  0.707107+0.j]))
(array([0.707107, 0.707107]), array([-0.707107+0.j,  0.707107+0.j]))
(array([0.707107, 0.707107]), array([ 0.+0.707107j, -0.-0.707107j]))

i=1
(array([0.707107, 0.707107]), array([-0.707107+0.j, -0.707107+0.j]))
(array([0.707107, 0.707107]), array([-0.707107+0.j,  0.707107+0.j]))
(array([0.707107, 0.707107]), array([ 0.707107+0.j, -0.707107+0.j]))
(array([0.707107, 0.707107]), array([0.+0.707107j, 0.-0.707107j]))

i=2
(array([0.707107, 0.707107]), array([-0.707107+0.j, -0.707107+0.j]))
(array([0.707107, 0.707107]), array([ 0.707107+0.j, -0.707107+0.j]))
(array([0.707107, 0.707107]), array([-0.707107+0.j,  0.707107+0.j]))
(array([0.707107, 0.707107]), array([-0.+0.707107j,  0.-0.707107j]))

i=3
(array([0.707107, 0.707107]), array([-0.707107+0.j, -0.707107+0.j]))
(array([0.707107, 0.707107]), array([-0.707107+0.j,  0.707107+0.j]))
(array([0.707107

##### Conclusions
* Similar to the B gauge case. Makes sense, as we have not changed anything about the right side of the SVD decomposition.

#### Eigenvalues and eigenvectors of left projected states

In [185]:
for i, t in enumerate(singular_and_eigen_vectors_symmetric_data):
    print("\ni={}".format(i))
    
    for j, l in enumerate(t):
        print(l[2])


i=0
(array([0.707107, 0.707107]), array([-0.707107+0.j, -0.707107+0.j]))
(array([0.707107, 0.707107]), array([-0.707107+0.j,  0.707107+0.j]))
(array([0.707107, 0.707107]), array([-0.707107+0.j,  0.707107+0.j]))
(array([0.707107, 0.707107]), array([-0.+0.707107j, -0.-0.707107j]))

i=1
(array([0.707107, 0.707107]), array([-0.707107+0.j, -0.707107+0.j]))
(array([0.707107, 0.707107]), array([-0.707107+0.j,  0.707107+0.j]))
(array([0.707107, 0.707107]), array([-0.707107+0.j,  0.707107+0.j]))
(array([0.707107, 0.707107]), array([0.+0.707107j, 0.-0.707107j]))

i=2
(array([0.707107, 0.707107]), array([-0.707107+0.j, -0.707107+0.j]))
(array([0.707107, 0.707107]), array([-0.707107+0.j,  0.707107+0.j]))
(array([0.707107, 0.707107]), array([-0.707107+0.j,  0.707107+0.j]))
(array([0.707107, 0.707107]), array([-0.+0.707107j, -0.-0.707107j]))

i=3
(array([0.707107, 0.707107]), array([-0.707107+0.j, -0.707107+0.j]))
(array([0.707107, 0.707107]), array([-0.707107+0.j,  0.707107+0.j]))
(array([0.707107

##### Conclusions
* Now we see behaviour similar to the right projected states. So it seems we can swap between these two behaviours by including/excluding the schmidt terms Should treat both sides uniformly. Which scheme is better?

#### Eigenvalues and eigenvectors of eigenvector

In [186]:
for i, t in enumerate(singular_and_eigen_vectors_symmetric_data):
    print("\ni={}".format(i))
    
    for j, l in enumerate(t):
        print(l[4])


i=0
(array([1., 1.]), array([ 1.+0.j, -1.+0.j]))
(array([0.876297, 0.206913]), array([0.371646+0.207839j, 0.371646-0.207839j]))
(array([1.016542, 0.443017]), array([-0.445894+0.j,  1.009983+0.j]))
(array([0.707136, 0.706415]), array([-0.706632+0.j,  0.706919+0.j]))

i=1
(array([], dtype=float64), array([], dtype=complex128))
(array([1.37568]), array([0.657746-1.e-06j]))
(array([0.867102]), array([0.706268-0.j]))
(array([0.906891]), array([ 0.000589+8.e-05j, -0.000589-8.e-05j]))

i=2
(array([], dtype=float64), array([], dtype=complex128))
(array([1.421103]), array([0.704459-1.e-06j]))
(array([1.019288]), array([-0.432556+1.5e-05j]))
(array([1.]), array([-0.001175-0.000174j,  0.001173+0.000175j]))

i=3
(array([0.000566]), array([], dtype=complex128))
(array([0.444666]), array([ 0.080632-0.000194j, -0.000178+0.000184j]))
(array([0.564675]), array([0.435556+1.e-06j]))
(array([0.80074]), array([-0.003487-2.6e-05j,  0.003481+2.5e-05j]))

i=4
(array([0.002803]), array([], dtype=complex128))


##### Conclusion
* Getting some interesting results in the trivial phase, with a dominant singular and eigenvalue of 1. Not sure how to make sense of non-trivial phase.

## Unitary operator transfer matrices

# Questions
* How do the Schmidt values interact with the eigenvalues of the transfer matrices? Do they correspond 1-1 to eigenvalues or something else? How do they affect the dominant eigenvalue etc?